In [ ]:
import tushare as ts

In [ ]:
ts.set_token('3463f0f3c5113c101cd9831597c579b4e429e5751f3f1739c6af830a')

In [ ]:
pro = ts.pro_api()

600519.SH

贵州茅台


In [ ]:
df1 = pro.daily(ts_code='600519.SH', end_date='20250722')

In [ ]:
df1

In [ ]:
type(df1)

In [ ]:
df1.to_csv('600519.SH.csv', index=False)

In [ ]:
ndsd = pro.daily(ts_code='300750.SZ', end_date='20250722')

In [ ]:
ndsd

In [ ]:
ndsd.to_csv('ndsd_300750.SZ.csv', index=False)

In [ ]:
joyson = pro.daily(ts_code='600699.SH', end_date='20250722')
joyson.to_csv('joyson_600699.SH.csv', index=False)

In [ ]:
tpjt = pro.daily(ts_code='601689.SH', end_date='20250722')
tpjt.to_csv('tpjt_601689.SH.csv', index=False)

In [ ]:
pinganbank = pro.daily(ts_code='000001.SZ', end_date='20250722')
pinganbank.to_csv('p_000001.SZ.csv', index=False)

In [ ]:
import akshare as ak

print(ak.__version__)

In [ ]:
import akshare as ak

stock_zh_a_hist_df, a1 = ak.stock_zh_a_hist(symbol="600699", period="daily", start_date="20250722", end_date='20250724', adjust="")

In [ ]:
stock_zh_a_hist_df

In [ ]:
a1

In [ ]:
joyson = pro.daily(ts_code='600699.SH', start_date="20250722", end_date='20250724',)

In [ ]:
joyson

In [ ]:
stock_zh_a_hist_df = ak.stock_zh_a_hist(symbol="600699", period="daily", start_date="20250722", end_date='20250724', adjust="")

In [ ]:
stock_hk = ak.stock_hk_hist_min_em()

In [ ]:
stock_hk

In [ ]:
stock_zh_a_hist_df = ak.stock_zh_a_hist(symbol="600699", period="daily", start_date="20000000", end_date='20250724', adjust="")

In [ ]:
stock_zh_a_hist_df

In [ ]:
joyson = pro.daily(ts_code='600699.SH', start_date="20000000", end_date='20250724',)

In [ ]:
joyson

In [ ]:
import pandas as pd
import torch
import numpy as np

def normalize_stock_data(df, features):
    """
    对股票数据进行Min-Max归一化，按股票代码分组处理
    
    参数:
    df: 包含多只股票数据的DataFrame，需包含'股票代码'列
    features: 需要归一化的特征列表
    
    返回:
    normalized_df: 归一化后的DataFrame
    scalers: 保存每只股票每个特征的min和max，用于后续反归一化
    """
    # 初始化存储归一化参数的字典
    scalers = {}
    
    # 复制原始数据，避免修改原数据
    normalized_df = df.copy()
    
    # 按股票代码分组处理
    for stock_code, group in df.groupby('股票代码'):
        scalers[stock_code] = {}
        
        # 对每个特征进行归一化
        for feature in features:
            # 获取该股票该特征的最小值和最大值
            min_val = group[feature].min()
            max_val = group[feature].max()
            
            # 处理极端情况（最大值等于最小值时避免除零）
            if max_val == min_val:
                normalized = np.zeros_like(group[feature].values)
            else:
                # Min-Max归一化公式: (x - min) / (max - min)
                normalized = (group[feature] - min_val) / (max_val - min_val)
            
            # 保存归一化参数
            scalers[stock_code][feature] = {
                'min': min_val,
                'max': max_val
            }
            
            # 更新归一化后的值
            normalized_df.loc[group.index, feature] = normalized
    
    # 将归一化后的数据转换为PyTorch张量（可选）
    tensor_data = torch.tensor(normalized_df[features].values, dtype=torch.float32)
    
    return normalized_df, scalers, tensor_data

# 示例用法
if __name__ == "__main__":
    # 1. 准备示例数据（实际使用时替换为你的数据）
    data = {
        '日期': ['2025-07-22', '2025-07-23', '2025-07-24', 
                '2025-07-22', '2025-07-23'],
        '开盘价': [17.71, 17.69, 17.95, 5.20, 5.30],
        '收盘价': [17.73, 17.92, 17.98, 5.15, 5.40],
        '最高价': [17.79, 18.12, 18.22, 5.30, 5.50],
        '最低价': [17.62, 17.60, 17.89, 5.10, 5.25],
        '成交量': [235707, 419966, 335495, 100000, 150000],
        '成交额': [416989247.0, 752988053.0, 604135187.0, 500000.0, 800000.0],
        '振幅': [0.96, 2.93, 1.84, 3.85, 4.76],
        '涨跌幅': [-0.28, 1.07, 0.33, -1.00, 4.85],
        '涨跌额': [-0.05, 0.19, 0.06, -0.05, 0.25],
        '股票代码': ['600699', '600699', '600699', '600000', '600000']
    }
    df = pd.DataFrame(data)
    
    # 2. 定义需要归一化的特征
    features_to_normalize = ['开盘价', '收盘价', '最高价', '最低价', 
                            '成交量', '成交额', '振幅', '涨跌幅', '涨跌额']
    
    # 3. 执行归一化
    normalized_df, scalers, tensor_data = normalize_stock_data(df, features_to_normalize)
    
    # 4. 输出结果
    print("归一化前的数据:\n", df[['股票代码'] + features_to_normalize].head())
    print("\n归一化后的数据:\n", normalized_df[['股票代码'] + features_to_normalize].head())
    print("\n归一化参数示例（600699的收盘价）:\n", scalers['600699']['收盘价'])
    print("\n转换为PyTorch张量的形状:", tensor_data.shape)


In [9]:
import akshare as ak

stock_zh_a_hist_df, a1 = ak.stock_zh_a_hist(symbol="600699", period="daily", start_date="20250522", end_date='20250724', adjust="")

In [10]:
stock_zh_a_hist_df

,日期,股票代码,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率
0,2025-05-22,600699,20.00,19.38,20.32,19.35,906208,1.782245e+09,4.83,-3.53,-0.71,6.62
1,2025-05-23,600699,19.58,19.23,20.04,19.11,721013,1.407556e+09,4.80,-0.77,-0.15,5.27
2,2025-05-26,600699,19.31,18.96,19.37,18.87,430418,8.185419e+08,2.60,-1.40,-0.27,3.15
3,2025-05-27,600699,18.90,18.51,19.00,18.23,426355,7.866479e+08,4.06,-2.37,-0.45,3.12
4,2025-05-28,600699,18.21,18.05,18.49,18.00,257017,4.685382e+08,2.65,-2.49,-0.46,1.88
5,2025-05-29,600699,18.05,18.37,18.84,18.05,389412,7.193108e+08,4.38,1.77,0.32,2.85
6,2025-05-30,600699,18.27,17.77,18.29,17.65,324766,5.785653e+08,3.48,-3.27,-0.60,2.37
7,2025-06-03,600699,17.58,17.62,17.88,17.50,239206,4.218507e+08,2.14,-0.84,-0.15,1.75
8,2025-06-04,600699,17.54,17.70,17.78,17.53,169222,2.992779e+08,1.42,0.45,0.08,1.24
9,2025-06-05,600699,17.76,17.87,17.92,17.57,224576,3.998541e+08,1.98,0.96,0.17,1.64


In [ ]:
# 2. 定义需要归一化的特征
features_to_normalize = ['开盘', '收盘', '最高', '最低', '振幅',]

# 3. 执行归一化
normalized_df, scalers, tensor_data = normalize_stock_data(stock_zh_a_hist_df, features_to_normalize)


In [ ]:
normalized_df

In [ ]:
tensor_data.shape

In [18]:
import pandas as pd
import numpy as np
class StockDataProcessor:
    def __init__(self, lookback=10):
        self.lookback = lookback  # 用过去10天的数据预测未来1天
        self.scalers = {}  # 存储每只股票的归一化参数
    
    def normalize(self, df, features):
        """对数据进行Min-Max归一化，按股票代码分组处理"""
        normalized_df = df.copy()
        
        for stock_code, group in df.groupby('ts_code'):
            self.scalers[stock_code] = {}
            for feature in features:
                min_val = group[feature].min()
                max_val = group[feature].max()
                if max_val == min_val:
                    normalized = np.zeros_like(group[feature].values)
                else:
                    normalized = (group[feature] - min_val) / (max_val - min_val)
                
                self.scalers[stock_code][feature] = {'min': min_val, 'max': max_val}
                normalized_df.loc[group.index, feature] = normalized
        
        return normalized_df
    
    def create_sequences(self, df, features, target_close_col, target_pct_col):
        """创建        创建序列数据
        输入: 过去lookback天的特征
        输出: 未来1天的收盘价(回归目标)和涨跌幅分类(分类目标)
        """
        sequences = []
        targets_reg = []  # 回归目标：收盘价
        targets_cls = []  # 分类目标：涨跌幅类别
        
        # 按股票代码分组处理
        for stock_code, group in df.groupby('ts_code'):
            group = group.sort_values('trade_date')  # 确保数据按日期排序
            values = group[features].values
            close_values = group[target_close_col].values
            pct_values = group[target_pct_col].values
            
            # 创建序列
            for i in range(len(values) - self.lookback):
                seq = values[i:i+self.lookback]
                # 回归目标：未来1天的收盘价
                target_reg = close_values[i+self.lookback]
                # 分类目标：未来1天的涨跌幅类别
                target_pct = pct_values[i+self.lookback]
                
                # 将涨跌幅分为三类：跌、平、涨
                if target_pct < -0.01:  # 跌幅超0.01%
                    target_cls = 0
                elif target_pct > 0.01:  # 涨幅超0.01%
                    target_cls = 2
                else:  # 平
                    target_cls = 1
                
                sequences.append(seq)
                targets_reg.append(target_reg)
                targets_cls.append(target_cls)
        
        return np.array(sequences), np.array(targets_reg), np.array(targets_cls)

In [ ]:
# 读取数据（替换为你的数据路径）
df = pd.read_csv('joyson_600699.SH.csv')

In [20]:
df

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,600699.SH,20250722,17.71,17.79,17.62,17.73,17.78,-0.05,-0.2812,235706.63,416989.247
1,600699.SH,20250721,17.85,17.86,17.66,17.78,17.78,0.00,0.0000,262598.95,465955.773
2,600699.SH,20250718,17.90,17.94,17.70,17.78,17.93,-0.15,-0.8366,290515.03,516084.376
3,600699.SH,20250717,17.88,17.96,17.68,17.93,17.88,0.05,0.2796,435530.10,777426.480
4,600699.SH,20250716,17.44,17.90,17.30,17.88,17.39,0.49,2.8177,556876.79,987651.066
...,...,...,...,...,...,...,...,...,...,...,...
5995,600699.SH,19981112,13.08,13.08,12.75,12.88,13.13,-0.25,-1.9000,7870.00,10146.919
5996,600699.SH,19981111,13.20,13.35,12.82,13.13,13.10,0.03,0.2300,11330.00,14725.784
5997,600699.SH,19981110,13.10,13.45,12.87,13.10,13.15,-0.05,-0.3800,10379.00,13467.110
5998,600699.SH,19981109,13.23,13.60,12.93,13.15,13.30,-0.15,-1.1300,22047.00,28790.214


In [ ]:
# 读取数据（替换为你的数据路径）
df = pd.read_csv('joyson_600699.SH.csv')
# 定义特征列
features = ['open', 'close', 'high', 'low', 'vol', 
            'amount', 'pct_chg', 'change']

# 初始化数据处理器
lookback = 10  # 用过去10天数据预测未来1天
processor = StockDataProcessor(lookback=lookback)

# 归一化数据
normalized_df = processor.normalize(df, features)

# 创建序列数据
sequences, targets_reg, targets_cls = processor.create_sequences(
    normalized_df, 
    features, 
    target_close_col='close', 
    target_pct_col='pct_chg'
)


In [22]:
sequences

array([[[0.2140255 , 0.20572775, 0.21880094, ..., 0.0055233 ,
         0.70128381, 0.47346939],
        [0.20637523, 0.20300888, 0.21065024, ..., 0.0068581 ,
         0.74071527, 0.49795918],
        [0.20400729, 0.20210259, 0.20793335, ..., 0.00320622,
         0.75790922, 0.50816327],
        ...,
        [0.22367942, 0.18887076, 0.22242347, ..., 0.00717164,
         0.66231087, 0.45306122],
        [0.19125683, 0.18198296, 0.19308096, ..., 0.00604209,
         0.69624026, 0.4744898 ],
        [0.18378871, 0.17328258, 0.18166999, ..., 0.00465853,
         0.67491976, 0.46428571]],

       [[0.20637523, 0.20300888, 0.21065024, ..., 0.0068581 ,
         0.74071527, 0.49795918],
        [0.20400729, 0.20210259, 0.20793335, ..., 0.00320622,
         0.75790922, 0.50816327],
        [0.20582878, 0.20264637, 0.20612208, ..., 0.00350619,
         0.77189363, 0.51632653],
        ...,
        [0.19125683, 0.18198296, 0.19308096, ..., 0.00604209,
         0.69624026, 0.4744898 ],
        [0.1